In [56]:
!pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 1.1 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install feature-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 4.8 MB/s eta 0:00:00ta 0:00:01


In [57]:
# import liberaries
import os
import boto3

import pickle
import numpy as np
import xgboost as xgb
import pandas as pd

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
	OneHotEncoder,
	OrdinalEncoder,
	StandardScaler,
	MinMaxScaler,
	PowerTransformer,
	FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
	RareLabelEncoder,
	MeanEncoder,
	CountFrequencyEncoder
)

import matplotlib.pyplot as plt

import warnings
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import(
  IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner

)

In [5]:
pd.set_option("display.max_columns", None)

In [6]:
sklearn.set_config(transform_output="pandas")


In [7]:

warnings.filterwarnings("ignore")

In [15]:
train=pd.read_csv('train.csv')
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-21,Kolkata,Banglore,21:10:00,16:20:00,19h 10m,1.0,In-flight meal not included,10844
1,Air India,2019-03-18,Banglore,New Delhi,11:05:00,22:10:00,11h 5m,2.0,No Info,10846
2,Jet Airways,2019-05-18,Kolkata,Banglore,08:25:00,16:20:00,7h 55m,1.0,No Info,14781
3,Air India,2019-06-03,Delhi,Cochin,10:00:00,19:15:00,9h 15m,1.0,No Info,10979
4,Air India,2019-06-09,Banglore,Delhi,06:10:00,08:55:00,2h 45m,0.0,No Info,6961
...,...,...,...,...,...,...,...,...,...,...
635,Jet Airways,2019-06-27,Delhi,Cochin,23:05:00,12:35:00,13h 30m,2.0,In-flight meal not included,11150
636,Jet Airways,2019-06-06,Kolkata,Banglore,06:30:00,18:15:00,11h 45m,1.0,In-flight meal not included,10844
637,Jet Airways,2019-05-06,Kolkata,Banglore,21:10:00,09:20:00,12h 10m,1.0,In-flight meal not included,9663
638,Multiple Carriers,2019-06-24,Delhi,Cochin,11:25:00,19:15:00,7h 50m,1.0,No Info,7155


In [10]:
val=pd.read_csv('val.csv')
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-01,Kolkata,Banglore,16:30:00,08:15:00,15h 45m,1.0,No Info,13941
1,Indigo,2019-03-06,Chennai,Kolkata,14:40:00,17:05:00,2h 25m,0.0,No Info,7295
2,Multiple Carriers,2019-06-01,Delhi,Cochin,13:00:00,01:30:00,12h 30m,1.0,No Info,7723
3,Indigo,2019-05-06,Banglore,Delhi,01:30:00,04:15:00,2h 45m,0.0,No Info,4284
4,Air India,2019-04-03,Kolkata,Banglore,20:45:00,23:20:00,2h 35m,0.0,No Info,4460
...,...,...,...,...,...,...,...,...,...,...
155,Jet Airways,2019-06-15,Delhi,Cochin,06:45:00,04:25:00,21h 40m,2.0,No Info,13376
156,Indigo,2019-04-24,Delhi,Cochin,05:05:00,16:10:00,11h 5m,1.0,No Info,6171
157,Jet Airways,2019-03-12,Banglore,New Delhi,21:25:00,07:40:00,10h 15m,1.0,No Info,11717
158,Jet Airways,2019-06-12,Kolkata,Banglore,21:10:00,04:40:00,7h 30m,1.0,In-flight meal not included,7594


In [11]:
test=pd.read_csv('test.csv')
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-03-06,Delhi,Cochin,13:00:00,21:00:00,8h,1.0,In-flight meal not included,15136
1,Indigo,2019-06-15,Delhi,Cochin,16:00:00,21:00:00,5h,1.0,No Info,6069
2,Jet Airways,2019-06-24,Banglore,Delhi,08:20:00,11:20:00,3h,0.0,In-flight meal not included,5769
3,Air India,2019-06-12,Delhi,Cochin,05:00:00,19:15:00,38h 15m,2.0,No Info,9443
4,Multiple Carriers,2019-06-24,Delhi,Cochin,13:15:00,22:30:00,9h 15m,1.0,No Info,8018
...,...,...,...,...,...,...,...,...,...,...
195,Jet Airways,2019-05-09,Kolkata,Banglore,16:30:00,18:15:00,25h 45m,1.0,No Info,13941
196,Jet Airways,2019-05-09,Kolkata,Banglore,09:35:00,21:05:00,11h 30m,1.0,In-flight meal not included,12384
197,Vistara,2019-05-21,Kolkata,Banglore,17:00:00,20:20:00,27h 20m,1.0,No Info,9345
198,Indigo,2019-03-09,Banglore,New Delhi,20:00:00,22:55:00,2h 55m,0.0,No Info,6144


In [16]:
train.duration

0      19h 10m
1       11h 5m
2       7h 55m
3       9h 15m
4       2h 45m
        ...   
635    13h 30m
636    11h 45m
637    12h 10m
638     7h 50m
639    23h 40m
Name: duration, Length: 640, dtype: object

In [17]:
def convert_duration_to_minutes(duration_series):
    def convert_single_duration(duration_str):
        hours, minutes = 0, 0
        if 'h' in duration_str:
            hours = int(duration_str.split('h')[0].strip())
            duration_str = duration_str.split('h')[1].strip()
        if 'm' in duration_str:
            minutes = int(duration_str.split('m')[0].strip())
        total_minutes = hours * 60 + minutes
        return total_minutes
    
    return duration_series.apply(convert_single_duration).astype(int)

In [18]:
train['duration']=convert_duration_to_minutes(train['duration'])

In [19]:
train.dtypes

airline             object
date_of_journey     object
source              object
destination         object
dep_time            object
arrival_time        object
duration             int64
total_stops        float64
additional_info     object
price                int64
dtype: object

In [45]:
train['dep_time']

0      21:10:00
1      11:05:00
2      08:25:00
3      10:00:00
4      06:10:00
         ...   
635    23:05:00
636    06:30:00
637    21:10:00
638    11:25:00
639    08:00:00
Name: dep_time, Length: 640, dtype: object

# preprocessing Operations

In [20]:

# airline
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

#doj
feature_to_extract = ["month", "week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
    ("scaler", MinMaxScaler())
])

# source & destination
location_pipe1 = Pipeline(steps=[
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", MeanEncoder()),
    ("scaler", PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
    return (
        X
        .assign(**{
            f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

location_transformer = FeatureUnion(transformer_list=[
    ("part1", location_pipe1),
    ("part2", FunctionTransformer(func=is_north))
])

# dep_time & arrival_time
time_pipe1 = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
    ("scaler", MinMaxScaler())
])

def part_of_day(X, morning=4, noon=12, eve=16, night=20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col: pd.to_datetime(X.loc[:, col]).dt.hour
        for col in columns
    })

    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day": np.select(
                [X_temp.loc[:, col].between(morning, noon, inclusive="left"),
                 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
                 X_temp.loc[:, col].between(eve, night, inclusive="left")],
                ["morning", "afternoon", "evening"],
                default="night"
            )
            for col in columns
        })
        .drop(columns=columns)
    )

time_pipe2 = Pipeline(steps=[
    ("part", FunctionTransformer(func=part_of_day)),
    ("encoder", CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list=[
    ("part1", time_pipe1),
    ("part2", time_pipe2)
])

# duration
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma


    def fit(self, X, y=None):
        if not self.variables:
            self.variables = X.select_dtypes(include="number").columns.to_list()

        self.reference_values_ = {
            col: (
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values
                .reshape(-1, 1)
            )
            for col in self.variables
        }

        return self


    def transform(self, X):
        objects = []
        for col in self.variables:
            columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
            obj = pd.DataFrame(
                data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
                columns=columns
            )
            objects.append(obj)
        return pd.concat(objects, axis=1)
    

def duration_category(X, short=180, med=400):
    return (
        X
        .assign(duration_cat=np.select([X.duration.lt(short),
                                    X.duration.between(short, med, inclusive="left")],
                                    ["short", "medium"],
                                    default="long"))
        .drop(columns="duration")
    )

def is_over(X, value=1000):
    return (
        X
        .assign(**{
            f"duration_over_{value}": X.duration.ge(value).astype(int)
        })
        .drop(columns="duration")
    )

duration_pipe1 = Pipeline(steps=[
    ("rbf", RBFPercentileSimilarity()),
    ("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
    ("cat", FunctionTransformer(func=duration_category)),
    ("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
    ("part1", duration_pipe1),
    ("part2", duration_pipe2),
    ("part3", FunctionTransformer(func=is_over)),
    ("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
    ("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
    ("imputer", SimpleImputer(strategy="median")),
    ("union", duration_union)
])

# total_stops
def is_direct(X):
    return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("", FunctionTransformer(func=is_direct))
])

# additional_info
info_pipe1 = Pipeline(steps=[
    ("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
    return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

info_union = FeatureUnion(transformer_list=[
("part1", info_pipe1),
("part2", FunctionTransformer(func=have_info))
])

info_transformer = Pipeline(steps=[
("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
("union", info_union)
])

# column transformer
column_transformer = ColumnTransformer(transformers=[
("air", air_transformer, ["airline"]),
("doj", doj_transformer, ["date_of_journey"]),
("location", location_transformer, ["source", 'destination']),
("time", time_transformer, ["dep_time", "arrival_time"]),
("dur", duration_transformer, ["duration"]),
("stops", total_stops_transformer, ["total_stops"]),
("info", info_transformer, ["additional_info"])
], remainder="passthrough")

# feature selector
estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selector = SelectBySingleFeaturePerformance(
estimator=estimator,
scoring="r2",
threshold=0.1
) 

# preprocessor
preprocessor = Pipeline(steps=[
("ct", column_transformer),
("selector", selector)
])

In [21]:
preprocessor.fit(
    train.drop(columns="price"),
    train.price.copy()
)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt',
                                                                   Datetime...
                                                                                                                                     replace_with='Other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x7f68e0e88c10>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [22]:
preprocessor.transform(train.drop(columns='price'))

,air__airline_Indigo,air__airline_Jet Airways,air__airline_Other,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,location__destination,dur__duration_rbf_25,dur__duration_cat,dur__duration_over_1000,dur__duration,stops__total_stops,stops__is_direct_flight
0,0.0,1.0,0.0,0.705882,0.686441,0.022464,-0.001909,-0.363278,2.0,1,0.965485,1.0,0
1,0.0,0.0,0.0,0.176471,0.144068,-1.107829,-0.992687,-0.363278,2.0,0,0.009974,2.0,0
2,0.0,1.0,0.0,0.647059,0.661017,0.022464,-0.001909,-0.363278,2.0,0,-0.364350,1.0,0
3,0.0,0.0,0.0,0.823529,0.796610,1.007938,1.007896,-0.363278,2.0,0,-0.206740,1.0,0
4,0.0,0.0,0.0,0.823529,0.847458,-1.107829,-1.800728,-0.359499,0.0,0,-0.975090,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,0.0,1.0,0.0,1.000000,1.000000,1.007938,1.007896,-0.363278,2.0,0,0.295642,2.0,0
636,0.0,1.0,0.0,0.823529,0.822034,0.022464,-0.001909,-0.363278,2.0,0,0.088779,1.0,0
637,0.0,1.0,0.0,0.588235,0.559322,0.022464,-0.001909,-0.363278,2.0,0,0.138032,1.0,0
638,0.0,0.0,0.0,1.000000,0.974576,1.007938,1.007896,-0.363278,2.0,0,-0.374201,1.0,0


# we will transform our datasubsets and pass to the s3 bucket

In [23]:
# sagemaker-flights-price-pred-bucket

NameError: name 'flights' is not defined

In [26]:
#willpreprocess our data and pass to bucket
BUCKET_NAME="sagemaker-flights-price-pred-bucket"

DATA_PREFIX='data'

In [27]:
def  get_file_name(name):
    return f"{name}-pre.csv"

In [28]:
# see we want to send our data sets to bucket befor sending we will export them here
# and befor ex[prt we will preprocess them too and then send to bucket]
def export_data(data, name, pre):
    X=data.drop(columns='price')
    y=data.price.copy()
    # first split the data
    # now we will preproces
    X_pre=pre.transform(X)
    
    # exporting
    file_name=get_file_name(name)
    (
    y
    .to_frame()
    .join(X_pre)
    .to_csv(file_name, index=False, header=False)
        # see ab hmare pass asa data ajauyga jo we can send to sagemaker
        # bcoz sagemake want data in this format
    
    )

    

In [29]:
# sagemake wantsdata in different format
# target columns should be at 0th index and rest other columns
y_tr=train.price.copy()
X_tr=train.drop(columns='price')

(
 y_tr
    .to_frame()
    .join(X_tr)
)

,price,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info
0,10844,Jet Airways,2019-05-21,Kolkata,Banglore,21:10:00,16:20:00,1150,1.0,In-flight meal not included
1,10846,Air India,2019-03-18,Banglore,New Delhi,11:05:00,22:10:00,665,2.0,No Info
2,14781,Jet Airways,2019-05-18,Kolkata,Banglore,08:25:00,16:20:00,475,1.0,No Info
3,10979,Air India,2019-06-03,Delhi,Cochin,10:00:00,19:15:00,555,1.0,No Info
4,6961,Air India,2019-06-09,Banglore,Delhi,06:10:00,08:55:00,165,0.0,No Info
...,...,...,...,...,...,...,...,...,...,...
635,11150,Jet Airways,2019-06-27,Delhi,Cochin,23:05:00,12:35:00,810,2.0,In-flight meal not included
636,10844,Jet Airways,2019-06-06,Kolkata,Banglore,06:30:00,18:15:00,705,1.0,In-flight meal not included
637,9663,Jet Airways,2019-05-06,Kolkata,Banglore,21:10:00,09:20:00,730,1.0,In-flight meal not included
638,7155,Multiple Carriers,2019-06-24,Delhi,Cochin,11:25:00,19:15:00,470,1.0,No Info


In [30]:
def upload_to_bucket(name):
    file_name = get_file_name(name)
    
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(DATA_PREFIX, f"{name}/{name}.csv"))
        .upload_file(file_name)
    )

In [55]:
# def upload_to_bucket(name):
#     file_name=get_file_name(name)
#     (
#     boto3
#         .session()
#         .resource('s3')# want s3 bucket so resource m bvo diya
#         .bucket(BUCKET_NAME)
#         .Object(os.path.join(DATA_PREFIX, f"{name}/{name}.csv"))
#         .upload_file(file_name)
#         # see ye kya kr rha hau  buckt k andar
#         # it will make Data  uske andar naam let say train and inside train it wilkk
#         # give train.csv path=data/train/train.csv
#     )
    
    # this is the code to send code in s3 buckets

In [31]:
def export_and_upload_inBucket(data, name, pre):
    export_data(data, name, pre)
    upload_to_bucket(name)

In [32]:
export_and_upload_inBucket(train, 'train', preprocessor)

In [33]:
pd.read_csv('train-pre.csv')

,10844,0.0,1.0,0.0.1,0.7058823529411765,0.6864406779661016,0.022463720690047966,-0.0019094382701410906,-0.3632778254381368,2.0,1,0.9654853432176175,1.0.1,0
0,10846,0.0,0.0,0.0,0.176471,0.144068,-1.107829,-0.992687,-0.363278,2.0,0,0.009974,2.0,0
1,14781,0.0,1.0,0.0,0.647059,0.661017,0.022464,-0.001909,-0.363278,2.0,0,-0.364350,1.0,0
2,10979,0.0,0.0,0.0,0.823529,0.796610,1.007938,1.007896,-0.363278,2.0,0,-0.206740,1.0,0
3,6961,0.0,0.0,0.0,0.823529,0.847458,-1.107829,-1.800728,-0.359499,0.0,0,-0.975090,0.0,1
4,6961,0.0,0.0,0.0,0.941176,0.923729,-1.107829,-1.800728,2.565923,0.0,0,-0.965239,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,11150,0.0,1.0,0.0,1.000000,1.000000,1.007938,1.007896,-0.363278,2.0,0,0.295642,2.0,0
635,10844,0.0,1.0,0.0,0.823529,0.822034,0.022464,-0.001909,-0.363278,2.0,0,0.088779,1.0,0
636,9663,0.0,1.0,0.0,0.588235,0.559322,0.022464,-0.001909,-0.363278,2.0,0,0.138032,1.0,0
637,7155,0.0,0.0,0.0,1.000000,0.974576,1.007938,1.007896,-0.363278,2.0,0,-0.374201,1.0,0


In [36]:
export_and_upload_inBucket(val, 'val', preprocessor)

In [37]:
val.dtypes

airline             object
date_of_journey     object
source              object
destination         object
dep_time            object
arrival_time        object
duration             int64
total_stops        float64
additional_info     object
price                int64
dtype: object

In [68]:
print(X_train.dtypes)
print(val.dtypes)

airline             object
date_of_journey     object
source              object
destination         object
dep_time            object
arrival_time        object
duration            object
total_stops        float64
additional_info     object
dtype: object
airline             object
date_of_journey     object
source              object
destination         object
dep_time            object
arrival_time        object
duration            object
total_stops        float64
additional_info     object
price                int64
dtype: object


In [24]:
print(val.isna().sum())


airline            0
date_of_journey    0
source             0
destination        0
dep_time           0
arrival_time       0
duration           0
total_stops        0
additional_info    0
price              0
dtype: int64


In [35]:
val['duration']=convert_duration_to_minutes(val['duration'])

In [38]:
test['duration']=convert_duration_to_minutes(test['duration'])

In [39]:
export_and_upload_inBucket(test, 'test', preprocessor)

# Model and  HyperParameter Tuning set up

In [47]:
session=sagemaker.session
# region_name=session.boto_region_name

In [46]:
!pip install --upgrade sagemaker boto3


In [59]:
import boto3
import sagemaker

In [53]:
# Get the region name
region_name = boto3.Session().region_name

print("Region name:", region_name)

Region name: us-east-1


In [60]:
# Create a SageMaker session
sagemaker_session = sagemaker.Session()



In [61]:
region_name = boto3.Session().region_name


In [ ]:
# why region name?
# sagemaker builtin alg need this session and name to work

In [55]:
# now the model we will malke we need tpo save that in to our s3Bucket
output_path=f"s3://{BUCKET_NAME}/model/output"

In [63]:
model = sagemaker.estimator.Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,  # run max for 300 sec
    max_wait=600,  # max wait for 600 sec to help save cost
    sagemaker_session=sagemaker_session  # Pass the correct session object
)

print("Estimator created successfully.")

Estimator created successfully.


In [64]:
# model = Estimator(
#     image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
#     # this we are telling to sagemaker we  gonna use xgboosgt
#     role=sagemaker.get_execution_role(),
#     #this role is IAM role it will use that IAM role
#     instance_count=1,
#     #instance_count--1 means we need one ec2
#     instance_type="ml.m4.xlarge",
#     volume_size=5,
#     output_path=output_path,
#     use_spot_instances=True,
#     max_run=300,# run max for 300 sec
#     max_wait=600,# max w8 for 600 sec it will help us in saving cost
#     sagemaker_session=session
# )

In [65]:
model.set_hyperparameters(
    objective="reg:linear",# reg means means squared error
    num_round=10,#num of trees
    eta=0.1,# lr
    max_depth=5,
    subsample=0.8,# for each tree give 80percent of the rows randomly
    colsample_bytree=0.8, # for each tre give 80 percent of thre random columns
    alpha=0.1
)

In [66]:
# we created a dict
# we will tune the values
hyperparameter_ranges = {
    "eta": ContinuousParameter(0.05, 0.2),
    # eta is lr which takes floating values so we give comtimousparameter
    "alpha": ContinuousParameter(0, 1),
    # same for alpha  regularization
    "max_depth": IntegerParameter(3, 5)
    # we can use moreparameters but to save cost i am using 3
}

In [67]:

tuner = HyperparameterTuner(
    estimator=model,
    objective_metric_name="validation:rmse",
    hyperparameter_ranges=hyperparameter_ranges,
    strategy="Bayesian",
    objective_type="Minimize"
)
# see sagemaker  what it does it train on train dataset but evaluate on validation set



#  Data Channels

In [68]:
def get_data_channel(name):
    # see har ek data ka chanel hoga train, val, test
    bucket_path = f"s3://{BUCKET_NAME}/{DATA_PREFIX}/{name}"
    # will pass bucket path where data is present
    return TrainingInput(bucket_path, content_type="csv")
# uss location pr csv content ko we are fetching

In [69]:
# lets take out trainka data channel
train_data_channel = get_data_channel("train")
train_data_channel

In [70]:
val_data_channel = get_data_channel("val")

In [71]:
data_channels = {
    "train": train_data_channel,
    "validation": val_data_channel
}

# Now train and tune the model

In [72]:
tuner.fit(data_channels)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


......................................!


In [ ]:
# we can deploy our model like on sagmaker
#tuner.best_estimator().deploy()
# but we want to deploy with streamlit so we will not do like this

# Model Evaluation

In [73]:
# see mne jo model bnaya tha bucket m use vha se slect krke download kiya
# no we have uploaded in to our project we will use that model for evaluation

with open("xgboost-model", "rb") as f:
    best_model = pickle.load(f)
 


In [74]:
best_model

In [83]:
from sklearn.metrics import r2_score

In [78]:

def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgb.DMatrix(data.iloc[:, 1:])
    # Xgboost data in this format
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [79]:

evaluate_model("train")

0.6092698571617328

In [80]:

evaluate_model("val")

0.5468659749338909

In [81]:

evaluate_model("test")

0.5408039509522444